In [ ]:
# pip install requests BeautifulSoup4 lxml
# pip install pygame
import requests
from bs4 import BeautifulSoup
import pandas as pd
import math
from tqdm import tqdm
import time
import os,  glob
import csv
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

# Для подключение к БД
import psycopg2                                    
host = '__.__.___.___'
database = '______'
dbuser = '________'
dbpassword = '_______'
port='_______'
from sqlalchemy import create_engine

greenplum = psycopg2.connect(host = host, database = database, user = dbuser, password = dbpassword)
gp_cursor = greenplum.cursor()

In [ ]:
# Оформление
%%html
<style>
.cell-output-ipywidget-background {
   background-color: transparent !important;
}
.jp-OutputArea-output {
   background-color: transparent;
}  
</style>

In [ ]:
# Подкдлючаемся к основной странице
url = 'https://www.auchan.ru/catalog'
response = requests.get(url, verify=False)
response.raise_for_status()
soup = BeautifulSoup(response.text, 'lxml')
print(soup.prettify())

# Смотрим на основные категории

In [ ]:
url = 'https://www.auchan.ru'
main_links, folder_name = [], []

for link in soup.find_all('__','_________________________________'):
    main_links.append(url+link.get('href'))
    folder_name.append(link.get('href').replace('/catalog/',"").replace('/',"").replace('-',"_"))

print(folder_name)

# Достаем группы товаров из категорий

In [ ]:
start_time = time.time()
category_links = []
leng = len(main_links)
links = 0
progress = IntProgress(min = 0, max = len(main_links), value = links, style={'bar_color': '#f08080'}, layout=Layout(width='20%', height='20px'))
display(progress)

for i in main_links:
    try:
        response = requests.get(i, verify=False)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'lxml')

        for link in soup.find_all('___','________________________________________________'):
            category_links.append(url+link.get('href'))
        
        links = links + 1    
        perc = round((links / leng) * 100, 2)
        end_time = time.time()
        predict = round((((end_time - start_time)/links)*(leng-links))/60,2)
        print (' PROCESS: {}/{}  ||  {} %  ||  WAITING TIME {} MIN     '.format(links,leng,perc,predict), end='\r')
    
    except Exception as e:
        print ('SOMETHING IS WRONG WITH {}'.format(link.get('href')))
        pass

    output = wi.Output()
    progress.value = links

# Добавляем номера страниц

In [ ]:
start_time = time.time()
numbers = []
leng = len(category_links)
links = 0
progress = IntProgress(min = 0, max = len(category_links), value = links, style={'bar_color': '#f08080'}, layout=Layout(width='20%', height='20px'))
display(progress)

for category in category_links:
    try:
        response = requests.get(category, verify=False)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'lxml')

        f = 1
        n = soup.find('_____','_______________').text                     # достаем кол-во товаров в подкатегории
        n = n.replace('(',"").replace(')', "")
        k = math.ceil(int(n)/40)

        while f <= k:
            numbers.append(category+'?page={}'.format(f))
            f = f + 1

        links = links + 1
        perc = round((links / leng) * 100, 2)
        end_time = time.time()
        predict = round((((end_time - start_time)/links)*(leng-links))/60,2)
        print (' PROCESS: {}/{}  ||  {} %  ||  WAITING TIME {} MIN     '.format(links,leng,perc,predict), end='\r')
        
    except Exception as e:
        print ('SOMETHING IS WRONG WITH {}      '.format(category), end='\r')
        pass
    
    output = wi.Output()
    progress.value = links

# Собираем страницы продуктов

In [ ]:
start_time = time.time()
full_path_art = []
leng = len(numbers)
links = 0
progress = IntProgress(min = 0, max = len(numbers), value = links, style={'bar_color': '#f08080'}, layout=Layout(width='20%', height='20px'))
display(progress)

for url_num in numbers:
    try:
        response = requests.get(url_num, verify=False)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'lxml')

        for link in soup.find_all('a','productCardPictureLink active css-of3y3a'):
            full_path_art.append(url_num+link.get('href'))
        
        links = links + 1
        perc = round((links / leng) * 100, 2)
        end_time = time.time()
        predict = round((((end_time - start_time)/links)*(leng-links))/60,2)
        print (' PROCESS: {}/{}  ||  {} %  ||  WAITING TIME {} MIN     '.format(links,leng,perc,predict), end='\r')
            
    except Exception as e:
        print (' SOMETHING IS WRONG WITH {}       '.format(link.get('href')), end='\r')
        pass
    
    output = wi.Output()
    progress.value = links

df_full_path = pd.DataFrame(full_path_art)
df_full_path.to_csv ("full_path.csv", index=False)

### Сохраняем полную информацию о ссылках на продукты

df_full_path = pd.DataFrame(full_path_art)
df_full_path.to_csv ("full_path.csv", index=False)

In [ ]:
path_prod=pd.read_csv('full_path.csv',names=['1','2','3','4','category','subcategory','5','product','name','6'],
encoding='cp1251',sep='/', skiprows=1)
path_prod = path_prod.drop(columns=['1','2','3','4','5','6'])
path_prod['path'] = 'https://www.auchan.ru/product/' + path_prod['name'] +'/'
path_prod.head()

# Преобразуем данные для разбиения

In [ ]:
path_prod=pd.read_csv('full_path.csv',names=['1','2','3','4','category','subcategory','5','product','name','6'],
encoding='cp1251',sep='/', skiprows=1)
path_prod = path_prod.drop(columns=['1','2','3','4','5','6'])
path_prod['path'] = 'https://www.auchan.ru/product/' + path_prod['name'] +'/'

cat = path_prod.category.unique()
files_name = []

for cat_name in cat:
    df = path_prod.loc[(path_prod['category'] == cat_name)]
    df = df.drop(columns=['category','subcategory','product','name'])
    file_name = "{}_part".format(cat_name)
    df.to_csv ('{}.csv'.format(file_name), index= False, header= False, sep=';')
    files_name.append(file_name)

prodycts_links = []
for i in files_name:
    prodycts_links.append(i+'.csv')

## Выбор категории для загрузки товаров

print(prodycts_links)

In [ ]:
file  = 'aziya_part.csv'        # выбрать категорию из списка выше
prod_cat=pd.read_csv(file, names=['links'])
prod_cat = prod_cat['links'].to_list()

# Собираем имя товара и артикул

In [ ]:
start_time = time.time()
name_art = []
links = 0
leng = len(prod_cat)
progress = IntProgress(min = 0, max = leng, value = links, style={'bar_color': '#f08080'}, layout=Layout(width='20%', height='20px'))
display(progress)      

for file in prod_cat:
    try:
        response = requests.get(file, verify=False)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'lxml')

        name_art.append(file + soup.find('__','___________').text + '/'+ soup.find('td','css-1v23ygr').text)

        links = links + 1
        perc = round((links / leng) * 100, 2)
        end_time = time.time()
        predict = round((((end_time - start_time)/links)*(leng-links))/60,2)
        print (' PROCESS: {}/{}  ||  {} %  ||  WAITING TIME {} MIN    '.format(links,leng,perc,predict), end='\r')

    except Exception as e:
        print (' SOMETHING IS WRONG WITH {}           '.format(url), end='\r')
        pass

    output = wi.Output()
    progress.value = links

#print(folder_name)

In [ ]:
df = pd.DataFrame(name_art)
df.to_csv ("df_final.csv", index=False)       # УКАЗАТЬ ИМЯ ДЛЯ ИТОГОВОГО ФАЙЛА

df = pd.read_csv('df_final.csv')
df = df['0'].str.split('/', expand=True)
df.columns=['0','1','2','3','Имя_ENG','Наименование','Артикул','7']
df = df.drop(columns=['0','1','2','3','7'])
df.dtypes

## Объединение полученных данных с данными из Dbeaver

In [ ]:
query_prod = """
select sku, lvl_1, lvl_2, lvl_3, lvl_4 from products
"""

In [ ]:
products  =  pd.read_sql(query_prod, greenplum)
products['sku'] = products['sku'].astype(str)
products.dtypes

In [ ]:
df_merged = pd.merge(products, df, left_on='sku', right_on='Артикул')
df_merged = df_merged[['Артикул', 'Наименование', 'lvl_1', 'lvl_2', 'lvl_3','lvl_4']]
df_merged = df_merged.drop_duplicates ()
df_merged

### Запись артикулов и продуктов и загрузка в SAS

In [ ]:
df_art = pd.DataFrame(name_art)
df_art.to_csv ("prd_art.csv", index=False)

In [ ]:
# Для загрузки в базу
conn = create_engine('________________________________________________')

table_path = 'table_table'          #УКАЗАТЬ НАИМЕНОВАНИЕ ТАБЛИЦЫ В SAS
df_merged.to_sql(name=table_path, con=conn, schema='public', if_exists='replace')
print('Залито в базу, путь: {}'.format(table_path))